In [1]:
from stable_baselines3.ppo.ppo import PPO
import torch.nn as nn
import gym
import os
from panda import PandaEnv
import pybullet as p
from pyBullet import PyBullet
from task.Grasp.PandaGraspEnv import PandaGraspEnv
from task.wrapper import ProcessDepthFrame84, ProcessFrame84, DepthToPyTorch, ImageToPyTorch
# from PandaReachEnv import PandaReachEnv
import matplotlib.pyplot as plt
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_checker import check_env
import numpy as np
from custom_policy import CustomCNN, CustomActorCriticPolicy
from stable_baselines3.common.evaluation import evaluate_policy
from callback import CheckpointCallback, EvalCallback, SaveVecNormalizeCallback

In [2]:
env = PandaGraspEnv(sim = PyBullet(render =False))

env = ProcessFrame84(env)
# env = ImageToPyTorch(env)
'''
env = ProcessDepthFrame84(env)
# env = DepthToPyTorch(env)
'''
obs = env.reset()
print(obs.shape)


(64, 64, 3)


In [3]:
policy_kwargs = dict(
    features_extractor_class=CustomActorCriticPolicy,
    features_extractor_kwargs=dict(features_dim=128),
)
checkpoint_callback = CheckpointCallback(save_freq=1000, save_path='./log/',
                                         name_prefix='rl_model')
save_vec_normalize = SaveVecNormalizeCallback(
                save_freq=1, save_path='./log/Vec/')
eval_callback = EvalCallback(
                eval_env=env,
                callback_on_new_best=save_vec_normalize,
                best_model_save_path='./log/best_model/',
                n_eval_episodes=5,
                log_path='./log/Eval/',
                eval_freq=10000,
                deterministic=True,
            )

In [4]:
model = PPO(CustomActorCriticPolicy,env=env, verbose=1, tensorboard_log='/tmp/ppo/')

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [4]:
check_env(env)


robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2
robot: 0
table: 2


In [5]:
model = SAC('CnnPolicy', env=env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [ ]:
model.learn(10000, callback = checkpoint_callback)